In [1]:
import json
import pandas as pd
import autogen
from autogen import ConversableAgent, GroupChat, GroupChatManager
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from langchain_community.vectorstores import chroma

In [2]:
config_list=[{"model":"mistral", "api_type":"ollama"}]
llm_config={
    "timeout":600,
    "temperature":0.1,
    "cache_seed":42,
    "config_list":config_list,
}

In [3]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

In [4]:
def csv_to_json(filename):
    df=pd.read_csv(filename)
    normalized_logs=df.to_dict(orient="records")
    with open("normalized_logs.json", "w") as f:
        json.dump(normalized_logs, f, indent=4)
    return normalized_logs

In [5]:
alert_policy=csv_to_json("alert_policies.csv")
policy_definition=csv_to_json("Mock_Policy_Definitions.csv")
transaction_data=csv_to_json("Mock_Transactions.csv")

In [6]:
anomaly_detector=RetrieveUserProxyAgent(
    name="anomaly_detector",
    system_message='''You are a finance agent. You have an extra retrieval super power.
    You retrieve data from transaction_data and detect anomalies and suspicious pattern using llm intelligence.
    List all the anomalies and suspicious pattern you have detected.''',
    llm_config=llm_config,
    is_termination_msg=termination_msg,
    retrieve_config={
        "task":"qa",
        "docs_path":["Mock_Transactions.csv"],
        "docs_content":transaction_data,
        "chunk_token_size":100,
        "model":config_list[0]["model"],
        "vector_db":"chromadb",
        "get_or_create":True,},
    code_execution_config=False,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    default_auto_reply="Return 'TERMINATE' when you have gathered all the information.",
)
policy_validator_agent = RetrieveUserProxyAgent(
    name = "policy_validator_agent",
    system_message = '''You are finance agent. You have an extra retrieval super power.
    You gather anomalies and suspicious pattern from anomaly_detector and validate it with internal policy_definition document.
    Then classify the Severity with PolicyName and ComplianceReason using llm intelligence.
    (e.g., PolicyName: Sanctioned Countries Block, 
    Severity: Critical, 
    ComplianceReason: Transactions to sanctioned nations).''',
    retrieve_config={
        "task":"qa",
        "docs_path":["Mock_Policy_Definitions.csv"],
        "docs_content":policy_definition,
        "chunk_token_size":100,
        "model":config_list[0]["model"],
        "vector_db":"chromadb",
        "get_or_create":True,},
    llm_config = llm_config,
    code_execution_config = False,
    max_consecutive_auto_reply = 3,
    is_termination_msg = termination_msg,
    human_input_mode="NEVER",
    default_auto_reply="Return 'TERMINATE' when you have gathered all the information."
)
alerting_agent = RetrieveUserProxyAgent(
    name = "alerting_agent",
    system_message = '''You are finance agent. You have an extra retrieval super power.
    You gather data from anomaly_detector and policy_validator_agent. Based on the data you receive from your team of agents,
    you verify the internal alert_policy document and suggests response using llm intelligence.
    (e.g., AlertID: PolicyBreachDetected, 
    EscalationLevel: High
    MitigationAction: ApplyFraudLock, 
    NotifyTo: SecurityTeam).''',
    retrieve_config={
        "task":"qa",
        "docs_path":["alert_policies.csv"],
        "docs_content":alert_policy,
        "chunk_token_size":100,
        "model":config_list[0]["model"],
        "vector_db":"chromadb",
        "get_or_create":True,},
    llm_config = llm_config,
    code_execution_config = False,
    max_consecutive_auto_reply = 3,
    is_termination_msg = termination_msg,
    human_input_mode="NEVER",
    default_auto_reply="Return 'TERMINATE' when you have gathered all the information."
)
        

In [7]:
groupchat = GroupChat(
    agents=[anomaly_detector, policy_validator_agent, alerting_agent],
    messages=[],
    max_round=3,
    speaker_selection_method="round_robin"
)

In [8]:
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)
manager.initiate_chat(
    anomaly_detector,
    message="Detect if there is any anomaly and suspicious pattern from transaction data."
)

chat_manager (to anomaly_detector):

Detect if there is any anomaly and suspicious pattern from transaction data.

--------------------------------------------------------------------------------
anomaly_detector (to chat_manager):

1. Large and Sudden Increase in Transactions: If there's a sudden spike in the number of transactions or an unusually large amount being transacted, this could indicate fraudulent activity such as identity theft or credit card cloning.

2. Unusual Geographical Pattern: Transactions happening from multiple locations in a short period that are not normally associated with the account or user's location. This could suggest that the account has been compromised and is being used by someone else.

3. Frequent Small Transactions: A series of small, frequent transactions instead of larger, less frequent ones may indicate that the user is trying to avoid detection limits or suspicion.

4. Infrequently Active Account with Multiple Transactions: If an account that ha

ChatResult(chat_id=None, chat_history=[{'content': 'Detect if there is any anomaly and suspicious pattern from transaction data.', 'role': 'assistant', 'name': 'chat_manager'}, {'content': "1. Large and Sudden Increase in Transactions: If there's a sudden spike in the number of transactions or an unusually large amount being transacted, this could indicate fraudulent activity such as identity theft or credit card cloning.\n\n2. Unusual Geographical Pattern: Transactions happening from multiple locations in a short period that are not normally associated with the account or user's location. This could suggest that the account has been compromised and is being used by someone else.\n\n3. Frequent Small Transactions: A series of small, frequent transactions instead of larger, less frequent ones may indicate that the user is trying to avoid detection limits or suspicion.\n\n4. Infrequently Active Account with Multiple Transactions: If an account that has not been active for a long time sud